In [ ]:
#! pip install transformers

In [ ]:
import transformer_generate
from transformer_generate import init_model, generate, decode

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.datasets import IMDB
from torchtext.data import Field, LabelField, BucketIterator, TabularDataset

import spacy

import numpy as np

import random
import math
import time

from tqdm import tqdm

import json

SEED = 1234

random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

! python -m spacy download en

spacy_en = spacy.load('en')

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [ ]:
def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings (tokens)
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

SRC = Field(tokenize = tokenize_en,
            init_token = '<sos>',
            eos_token = '<eos>', 
            lower = True)

# should be 0 if the sentence is natural and 1 if this is encoded
LABEL = LabelField(dtype = torch.float)

In [ ]:
#take original dataset for first words
train_data, test_data = IMDB.splits(SRC, LABEL)

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

Generating data

In [ ]:
data_size = 1000  #size of generated dataset

# for first words - take first word from randomly chosen text from train_data
sent_ids = np.random.randint(len(train_data), size=data_size)

# bit lengths for bit stream to be encoded
bit_lenghts = np.random.choice([50, 100, 60, 70], data_size)

In [ ]:
#writing the record in json file 
def write_json(data, filename='data.json'): 
    with open(filename,'a') as f: 
        f.write(json.dumps(data)+"\n")

In [ ]:
#model inizialization
model, tokenizer = init_model()

In [ ]:
transformer_generate.TOKEN_COUNT

8

In [ ]:
#generating and writing to json file
for i, bit_len in tqdm(zip(sent_ids, bit_lenghts)):
    seq = list(np.random.binomial(1, 0.5, bit_len))
    text = generate(model, train_data.examples[i].text[0].capitalize(),
                  seq, tokenizer)

    write_json({"text": text, "label":0}, "../data/generated.json")

checking for decoding and encoding validity of a model

In [ ]:
seq = list(np.random.binomial(1, 0.5, 50))

text = generate(model, 'Hello', seq, tokenizer)

In [ ]:
text

"Hello I know the first one of the game's characters in-game"

In [ ]:
_tokens = tokenizer.encode(text, return_tensors='pt')
decoded_seq = decode(model, _tokens)

In [ ]:
np.array_equal(decoded_seq, seq)